## 收集数据
- tweet无法访问 :)
- 从github仓库[https://github.com/udacity/new-dand-advanced-china](https://github.com/udacity/new-dand-advanced-china)中下载文件

In [1]:
# -*- coding=utf-8 -*-
import requests

def download(url):
    with open(url.split('/')[-1], mode="wb") as f:
        respone = requests.get(url)
        f.write(respone.content)
        
# 下载 image-predictions.tsv
image_prediction_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv"
download(image_prediction_url)
# 下载 tweet_json.txt
tweet_json_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/tweet_json.txt"
download(tweet_json_url)

## 评估数据

In [81]:
# 使用pandas模块
import pandas as pd

In [150]:
# 读取和评估image-predictions.tsv数据
image_predictions = pd.read_csv("image-predictions.tsv", sep='\t')
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.0+ KB


In [151]:
image_predictions.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [152]:
# 处理和评估tweet_json.txt数据
import json
with open("tweet_json.txt", 'r') as json_file:
    for data in json_file.readlines():
        print(json.dumps(json.loads(data), indent=4, sort_keys=False, ensure_ascii=True))
        break

{
    "contributors": null, 
    "truncated": false, 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "id": 892420643555336193, 
    "favorite_count": 39492, 
    "full_text": "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU", 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "retweeted": false, 
    "coordinates": null, 
    "entities": {
        "symbols": [], 
        "user_mentions": [], 
        "hashtags": [], 
        "urls": [], 
        "media": [
            {
                "expanded_url": "https://twitter.com/dog_rates/status/892420643555336193/photo/1", 
                "display_url": "pic.twitter.com/MgUWQ76dJU", 
                "url": "https://t.co/MgUWQ76dJU", 
                "media_url_https": "https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg", 
                "id_str": "892420639486877696", 
                "sizes": {
    

In [156]:
# 逐行读取tweet_json.txt文件并添加到pandas DataFrame中，（至少）包含 tweet ID、retweet_count和favorite_count字段...
# 提出问题，放到list中，然后通过 'id_str': list1, 'fav_count': list2的方式创建df的效率是否更高？
df_tweet = pd.DataFrame()

# 观察tweet_json.txt后，对以下数据感兴趣：
# 'id_str', 'source', 'favorite_count', 'full_text', 'retweeted', 'retweet_count', 'created_at', 'expanded_url'
with open("tweet_json.txt", 'r') as json_file:
    indexs = ['id', 'source', 'favorite_count', 'full_text', 'retweeted', 'retweet_count', 'created_at']
    for data in json_file.readlines():
        dict_data = json.loads(data)
        
        data = []
        for idx in indexs:
            data.append(dict_data[idx])
        # print(type(dict_data['retweeted']));break
        # 获取media.expanded_url
        # print(dict_data['entities']);break;
        if dict_data['entities'].has_key('media'):
            data.append(dict_data['entities']['media'][0]['expanded_url'])
        else:
            data.append(None)
        
        s_tweet = pd.Series(data, index=indexs+['expanded_url'])
        df_tweet = df_tweet.append(s_tweet, ignore_index=True)

In [157]:
df_tweet.describe()

,favorite_count,id,retweet_count,retweeted
count,2352.000000,2.352000e+03,2352.000000,2352.0
mean,8109.198980,7.425913e+17,3134.932398,0.0
std,11980.795669,6.846210e+16,5237.846296,0.0
min,0.000000,6.660209e+17,0.000000,0.0
25%,1417.000000,6.783949e+17,618.000000,0.0
50%,3596.500000,7.193536e+17,1456.500000,0.0
75%,10118.000000,7.991219e+17,3628.750000,0.0
max,132318.000000,8.924206e+17,79116.000000,0.0


In [158]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 8 columns):
created_at        2352 non-null object
expanded_url      2073 non-null object
favorite_count    2352 non-null float64
full_text         2352 non-null object
id                2352 non-null float64
retweet_count     2352 non-null float64
retweeted         2352 non-null float64
source            2352 non-null object
dtypes: float64(4), object(4)
memory usage: 147.1+ KB


In [140]:
df_tweet

,created_at,expanded_url,favorite_count,full_text,id,retweet_count,retweeted,source
0,Tue Aug 01 16:23:56 +0000 2017,https://twitter.com/dog_rates/status/892420643...,39492.0,This is Phineas. He's a mystical boy. Only eve...,8.924206e+17,8842.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
1,Tue Aug 01 00:17:27 +0000 2017,https://twitter.com/dog_rates/status/892177421...,33786.0,This is Tilly. She's just checking pup on you....,8.921774e+17,6480.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
2,Mon Jul 31 00:18:03 +0000 2017,https://twitter.com/dog_rates/status/891815181...,25445.0,This is Archie. He is a rare Norwegian Pouncin...,8.918152e+17,4301.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
3,Sun Jul 30 15:58:51 +0000 2017,https://twitter.com/dog_rates/status/891689557...,42863.0,This is Darla. She commenced a snooze mid meal...,8.916896e+17,8925.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
4,Sat Jul 29 16:00:24 +0000 2017,https://twitter.com/dog_rates/status/891327558...,41016.0,This is Franklin. He would like you to stop ca...,8.913276e+17,9721.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
5,Sat Jul 29 00:08:17 +0000 2017,https://twitter.com/dog_rates/status/891087950...,20548.0,Here we have a majestic great white breaching ...,8.910880e+17,3240.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
6,Fri Jul 28 16:27:12 +0000 2017,https://twitter.com/dog_rates/status/890971913...,12053.0,Meet Jax. He enjoys ice cream so much he gets ...,8.909719e+17,2142.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
7,Fri Jul 28 00:22:40 +0000 2017,https://twitter.com/dog_rates/status/890729181...,66596.0,When you watch your owner call another dog a g...,8.907292e+17,19548.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
8,Thu Jul 27 16:25:51 +0000 2017,https://twitter.com/dog_rates/status/890609185...,28187.0,This is Zoey. She doesn't want to be one of th...,8.906092e+17,4403.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
9,Wed Jul 26 15:59:51 +0000 2017,https://twitter.com/dog_rates/status/890240255...,32467.0,This is Cassie. She is a college pup. Studying...,8.902403e+17,7684.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."


#### 质量
- id应该是字符串类型、favorite_count、retweet_count应该是int类型，retweeted应该是bool类型
- 只需要不包括转发的数据，即保留retweeted为False的数据
- 只需要包含图片的原始评级，即保留expanded_url为空的数据
- created_at应该是date类型
- 从full_text中提取分数时，分子为浮点数，分母为10
- 从full_text中提取分数时，如果分数出现多次，那么则取均值作为分数
- 从full_text中提取狗的名字

#### 清洁度
- 拆分分数，分别得出分子rating_numerator和分母rating_denominator
- source字段内容是html文本，只需要获取该html的text部分即可

## 清理

In [159]:
# 清理前的备份
df_tweet_clean = df_tweet.copy()

#### `id`: 这个是用来识别用户的，数值比较大，且是整数，应该是long类型
#### `favorite_count、retweet_count`: 可能用于计数比较，应该是int类型


##### 定义
对id转换成long类型：astype('long')
对favorite_count、id、retweet_count做类型转换：astype('int')

##### 代码

In [142]:
df_tweet_clean['id'] = df_tweet_clean['id'].astype(long)
df_tweet_clean['favorite_count'] = df_tweet_clean['favorite_count'].astype(int)
df_tweet_clean['retweet_count'] = df_tweet_clean['retweet_count'].astype(int)

##### 测试

In [143]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 8 columns):
created_at        2352 non-null object
expanded_url      2352 non-null object
favorite_count    2352 non-null int32
full_text         2352 non-null object
id                2352 non-null int64
retweet_count     2352 non-null int32
retweeted         2352 non-null float64
source            2352 non-null object
dtypes: float64(1), int32(2), int64(1), object(4)
memory usage: 128.7+ KB


#### `retweeted`: 应该是bool类型

##### 定义
对retweeted做类型转换：astype('bool')

##### 代码

In [144]:
df_tweet_clean['retweeted'] = df_tweet_clean['retweeted'].astype(bool)
df_tweet_clean.info()
#df_tweet_clean = df_tweet_clean[df_tweet_clean['retweeted'] == False]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 8 columns):
created_at        2352 non-null object
expanded_url      2352 non-null object
favorite_count    2352 non-null int32
full_text         2352 non-null object
id                2352 non-null int64
retweet_count     2352 non-null int32
retweeted         2352 non-null bool
source            2352 non-null object
dtypes: bool(1), int32(2), int64(1), object(4)
memory usage: 112.6+ KB


In [145]:
df_tweet_clean.retweeted.value_counts()

False    2352
Name: retweeted, dtype: int64

##### 测试
有上面可以知道，tweet_json.txt的数据都是原始评级的数据。

#### `expanded_url`: 应该包含图片的url

##### 定义
清理`expanded_url`为空的数据

##### 代码

In [173]:
df_tweet_clean = df_tweet_clean[df_tweet_clean['expanded_url'].notnull()]

##### 测试

In [174]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2073 entries, 0 to 2351
Data columns (total 8 columns):
created_at        2073 non-null object
expanded_url      2073 non-null object
favorite_count    2073 non-null float64
full_text         2073 non-null object
id                2073 non-null float64
retweet_count     2073 non-null float64
retweeted         2073 non-null float64
source            2073 non-null object
dtypes: float64(4), object(4)
memory usage: 145.8+ KB


In [147]:
# 把数据保存到 twitter_archive_master.csv
twitter_archive_master.to_csv('twitter_archive_master.csv', index=False)

NameError: name 'twitter_archive_master' is not defined